<a href="https://colab.research.google.com/github/vyshnavi-2/Sentiment-Analysis-using-ML/blob/main/Sentiment_Analysis_Using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#installing kaggle library
!pip install kaggle

Upload your kaggle.json file

In [15]:
#configuring the path of Kaggle.json.file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Import Twitter Sentiment dataset

In [16]:
#API to fetch data set and unzip it
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Download the dataset
api.dataset_download_files('kazanova/sentiment140', path='./sentiment140', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140


Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
import re #Regular Expression used for pattern matching and search through data
from nltk.corpus import stopwords #used for textual data natural processing
from nltk.stem.porter import PorterStemmer #used for redude the text into root words
from sklearn.feature_extraction.text import TfidfVectorizer #used for feature extraction
from sklearn.model_selection import train_test_split #used for splitting training and testing data
from sklearn.linear_model import LogisticRegression #used for training machine learning model
from sklearn.metrics import accuracy_score #calculates accuracy and performance
from transformers import pipeline, MarianMTModel, MarianTokenizer
import random
import torch
from torch import nn, optim


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
#printing the stopwords in English(words which are not required for processing)
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [23]:
#loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [24]:
#checking the number of rows and colums
twitter_data.shape


(1599999, 6)

In [25]:
#printing the first 5 rows of the dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [26]:
#naming the colums and reading the dataset again

column_names=['target','id','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')

In [27]:
#checking the number of rows and colums
twitter_data.shape

(1600000, 6)

In [28]:
#printing the first 5 rows of the dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [29]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [30]:
#checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Convert the target "4" to "1"

In [31]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [32]:
#checking the distribution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0 --> Negative Tweet

1 --> Positive Tweet

**Stemming**

Stemming is a process of reducing a word to its Root word

In [33]:
port_stem=PorterStemmer()

In [34]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  sremmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [35]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [36]:
twitter_data.head()


,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer you...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan i dive mani time ball manag save the ...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad i see


In [37]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer you...
1          upset updat facebook text might cri result sch...
2          kenichan i dive mani time ball manag save the ...
3                            whole bodi feel itchi like fire
4                            nationwideclass behav mad i see
                                 ...                        
1599995                 just woke have school best feel ever
1599996    thewdb com veri cool hear old walt interview h...
1599997                     are readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [38]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [39]:
#seperating the data and label
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [40]:
print(X)

['switchfoot http twitpic com zl awww bummer you shoulda got david carr third day d'
 'upset updat facebook text might cri result school today also blah'
 'kenichan i dive mani time ball manag save the rest go bound' ...
 'are readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [41]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to training data and test data

In [42]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)# stratify is used for equal distr of X&Y in train and test data

In [43]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [44]:
print(Y.shape,Y_train.shape,Y_test.shape)

(1600000,) (1280000,) (320000,)


In [45]:
#converting the textual data to numerical data

vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [46]:
print(X_train)

  (0, 436736)	0.27259876264838384
  (0, 354563)	0.3588091611460021
  (0, 185208)	0.5277679060576009
  (0, 109311)	0.3753708587402299
  (0, 235060)	0.41996827700291095
  (0, 443091)	0.4484755317023172
  (1, 160645)	1.0
  (2, 109311)	0.45915639201149566
  (2, 124490)	0.18923156633380322
  (2, 407323)	0.18710917798380342
  (2, 129417)	0.29076646660098543
  (2, 406421)	0.32108169269258585
  (2, 433582)	0.32968741386905526
  (2, 77931)	0.3128672120237835
  (2, 443455)	0.33488823001589063
  (2, 266745)	0.2412526672795364
  (2, 409165)	0.15170562659204556
  (2, 178072)	0.16191467578116914
  (2, 150722)	0.18805437673647912
  (2, 132317)	0.20291428205045584
  (2, 288488)	0.1673801447945276
  (3, 406421)	0.29029991238662284
  (3, 158719)	0.4456939372299574
  (3, 151777)	0.278559647704793
  (3, 56478)	0.5200465453608686
  :	:
  (1279996, 390152)	0.2206474219107611
  (1279996, 373166)	0.35212500999832036
  (1279996, 238092)	0.5249170684084672
  (1279996, 238093)	0.5606696159563151
  (1279997, 5685

In [47]:
print(X_test)

  (0, 15110)	0.1719352837797837
  (0, 31169)	0.1624772418052177
  (0, 67830)	0.26800375270827315
  (0, 106074)	0.36555450010904555
  (0, 132370)	0.255254889555786
  (0, 138171)	0.23688292264071406
  (0, 171389)	0.2805816206356074
  (0, 271032)	0.45356623916588285
  (0, 279098)	0.17825180109103442
  (0, 388370)	0.2198507607206174
  (0, 398928)	0.34910438732642673
  (0, 409165)	0.3143047059807971
  (0, 421006)	0.17915624523539805
  (1, 6463)	0.30733520460524466
  (1, 15110)	0.211037449588008
  (1, 145400)	0.575262969264869
  (1, 217577)	0.40288153995289894
  (1, 256792)	0.28751585696559306
  (1, 348155)	0.4739279595416274
  (1, 366225)	0.24595562404108307
  (2, 22532)	0.3532582957477176
  (2, 34402)	0.37916255084357414
  (2, 89451)	0.36340369428387626
  (2, 183325)	0.5892069252021465
  (2, 256849)	0.2564939661498776
  :	:
  (319995, 213339)	0.24848254898331648
  (319995, 232906)	0.23831341432205586
  (319995, 258593)	0.2735913557825173
  (319995, 296681)	0.369625344051842
  (319995, 3158

Training the machine Learning Model

Logistic Regression

In [48]:
model=LogisticRegression(max_iter=1000)

In [49]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy Score

In [50]:
#accuracy scre on the training data
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [51]:
print('Accuracy score on the training data:',training_data_accuracy)

Accuracy score on the training data: 0.8020359375


In [52]:
#accuracy scre on the training data
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [53]:
print('Accuracy score on the test data:',test_data_accuracy)

Accuracy score on the test data: 0.780575


Model accuracy=78%

Saving the trained model

In [8]:
import pickle

In [54]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

Using the Saved model for future predictions

In [55]:
#loading the saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [56]:
X_new=X_test[200]
print(Y_test[200])

prediction=loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [57]:
X_new=X_test[3]
print(Y_test[3])

prediction=loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

0
[0]
Negative Tweet
